# FERNANDO LEON FRANCO

In [1]:
import numpy as np
import polars as pl
from bs4 import BeautifulSoup
import os
from colorstreak import Logger
import re
from dataclasses import dataclass
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter

In [2]:
def print_bar(i, cantidad_registros, contexto="PROGRESO"):
    porcentaje = (i + 1) / cantidad_registros * 100
    # Con emojis
    barra = int(50 * (i + 1) / cantidad_registros) * "🟩"
    espacio = int(50 - len(barra)) * "⬛️"

    print(f"\r{contexto}: |{barra}{espacio}| {porcentaje:6.2f}%", end="", flush=True)

# MEJORAR LA CARGA DESDE UN CSV

In [3]:
def generar_csv(path_lectura, path_guardado):
    # Obtener la ruta absoluta del directorio actual
    # Leerlo y convertirlo en un DataFrame .csv para manipularlo mejor
    with open(path_lectura, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    data = [line.strip().split(":::") for line in lines]


    df = pl.DataFrame(data, schema=["id", "genero", "pais"], orient="row")


    # Guardar el DataFrame como un archivo .csv
    df.write_csv(path_guardado)
    return True

# ==================== Configuración de rutas =====================


base_path = os.getcwd()

# ===================== Creación de CSV de datos de prueba =====================
ruta_lectura_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth.txt")
ruta_guardado_prueba = os.path.join(base_path, "data/author_profiling/es_test/truth_test.csv")

creado = generar_csv(ruta_lectura_prueba, ruta_guardado_prueba)

if creado:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_prueba}"')
# ==============================================================================


# ===================== Creación de CSV de datos de entrenamiento ==============
ruta_lectura_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth.txt")
ruta_guardado_entrenamiento = os.path.join(base_path, "data/author_profiling/es_train/truth_train.csv")

creado_entrenamiento = generar_csv(ruta_lectura_entrenamiento, ruta_guardado_entrenamiento)
if creado_entrenamiento:
    Logger.info(f'Archivo CSV creado en: "{ruta_guardado_entrenamiento}"')
# ==============================================================================



[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_test/truth_test.csv"
[INFO] Archivo CSV creado en: "/Users/ferleon/Github/semestre_v/procesamiento_lenguaje/data/author_profiling/es_train/truth_train.csv"


# CARGAR LOS INDICES DESDE TRUTH DESDE EL CSV

In [4]:
def cargar_xml(id_archivo, train=True) -> str:
    ruta_base = os.path.join(base_path, "data/author_profiling/es_test" if not train else "data/author_profiling/es_train")
    ruta_archivo = os.path.join(ruta_base, f"{id_archivo}.xml")
    # Logger.info(f"ruta archivo: {ruta_archivo}")
    
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        xml_text = file.read()

    return xml_text

# ===================== Cargar CSV de datos  ==================
 
df_indices = pl.read_csv(ruta_guardado_entrenamiento)
print(df_indices.head())

cantidad_registros = len(df_indices)

registros_crudos = [("id_user","xml_doc","pais","genero") for i in range(cantidad_registros)]


for i, reg in enumerate(registros_crudos):
    id_archivo = df_indices['id'][i]
    id_user = id_archivo
    pais = df_indices['pais'][i]
    genero = df_indices['genero'][i]

    xml_text: str = cargar_xml(id_archivo)
    registros_crudos[i] = (id_user, xml_text, pais, genero)

    print_bar(i, cantidad_registros, contexto="CARGA XML")

print()
Logger.info("Carga de archivos XML completada.\n")
id_user, xml_text, pais, genero = registros_crudos[0]


Logger.debug(f"Total de tweets crudos cargados: {len(registros_crudos)} mostrando 1 registro:\n id:{id_user}\n pais:{pais}\n genero:{genero}\n xml_text:\n{xml_text[:50]}...\n")

shape: (5, 3)
┌─────────────────────────────────┬────────┬──────────┐
│ id                              ┆ genero ┆ pais     │
│ ---                             ┆ ---    ┆ ---      │
│ str                             ┆ str    ┆ str      │
╞═════════════════════════════════╪════════╪══════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ female ┆ colombia │
│ 4639c055f34ca1f944d0137a5aeb79… ┆ female ┆ colombia │
│ 92ffa98bade702b86417b118e8aca3… ┆ female ┆ colombia │
│ 4560c6567afcccef265f048ed117d0… ┆ female ┆ colombia │
│ 393866dfaa80d414c9896cf8723932… ┆ female ┆ colombia │
└─────────────────────────────────┴────────┴──────────┘
CARGA XML: |⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️⬛️|   0.14%

CARGA XML: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%
[INFO] Carga de archivos XML completada.

[DEBUG] Total de tweets crudos cargados: 4200 mostrando 1 registro:
 id:74bcc9b0882c8440716ff370494aea09
 pais:colombia
 genero:female
 xml_text:
<author lang="es">
	<documents>
		<document><![CDA...



# PROCESAMOS LOS TEXTOS XML Y LOS DEJAMOS EN UN DF

In [5]:
diccionario_idioma = {
    'es': 'Español', 
    'en': 'Inglés', 
    'fr': 'Francés', 
    'de': 'Alemán', 
    'it': 'Italiano', 
    'nl': 'Neerlandés'
}

def limpiar_xml(texto_xml:str):
    soup = BeautifulSoup(texto_xml, 'lxml-xml')   
    lang = str(soup.author.get('lang'))
    idioma = diccionario_idioma[lang] if lang in diccionario_idioma else lang
    
    documentos = soup.find_all('document') # Obtenemos todsa las etiquetas <document>
    tweets = [doc.get_text(separator=" ", strip=True) for doc in documentos] # Extraemos el texto de cada documento
    
    return tweets, idioma



# ===================== Procesamiento de limpieza del XML =====================
cantidad_registros = len(registros_crudos)

registros_procesados = []

for i, (id_user, doc_crudo, pais, genero) in enumerate(registros_crudos):
    lista_tweets_por_usuario, idioma = limpiar_xml(doc_crudo)
    
    for tweet in lista_tweets_por_usuario:
        registros_procesados.append({
            "id_user": id_user,
            "tweet_crudo": tweet,
            "pais": pais,
            "genero": genero,
            "idioma": idioma
        })
    print_bar(i, cantidad_registros, contexto="Progreso registros limpiados")

print()

# ===================== Creamos dataset persistente =====================

df_registros = pl.DataFrame(registros_procesados)
df_registros.write_parquet("data/author_profiling/registros_procesados.parquet")
Logger.info("Guardando archivo")

lf_registros = df_registros.lazy()

# ===================== Muestra =====================


muestra = lf_registros.limit(5).collect()
Logger.debug(f"Total de tweets procesados: {len(registros_procesados)} mostrando 5 primeros registros:\n {muestra}")


Progreso registros limpiados: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%
[INFO] Guardando archivo
[DEBUG] Total de tweets procesados: 419998 mostrando 5 primeros registros:
 shape: (5, 5)
┌─────────────────────────────────┬─────────────────────────────────┬──────────┬────────┬─────────┐
│ id_user                         ┆ tweet_crudo                     ┆ pais     ┆ genero ┆ idioma  │
│ ---                             ┆ ---                             ┆ ---      ┆ ---    ┆ ---     │
│ str                             ┆ str                             ┆ str      ┆ str    ┆ str     │
╞═════════════════════════════════╪═════════════════════════════════╪══════════╪════════╪═════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ Tiene que valer la pena que es… ┆ colombia ┆ female ┆ Español │
│ 74bcc9b0882c8440716ff370494aea… ┆ Tintas chinas, si ven ésto, es… ┆ colombia ┆ female ┆ Español │
│ 74bcc9b0882c8440716ff370494aea… ┆ "Maestro no le abrió!" -Ay qué… ┆ colombia ┆ female ┆ Espa


# Capa de dataclass de utilidd para normalización y limpieza de tweets

In [6]:
from nltk.tokenize import TweetTokenizer
import unicodedata
# ===================== Configuración de Limpieza de tweets =====================


@dataclass(frozen=True)
class ConfigLimpieza:
    normalizar_unicode: bool = True
    a_minusculas: bool = True
    quitar_urls: bool = True
    quitar_menciones: bool = True
    quitar_hashtags: bool = False 


    

@dataclass
class Tweet:
    URL = re.compile(r'https?://\S+', re.I)
    MENCION = re.compile(r'@\w+')
    HASHTAG = re.compile(r'#\w+')
    ESPACIO = re.compile(r'\s+')
    TT = TweetTokenizer()

    @staticmethod
    def limpiar(texto: str) -> str:
        cfg = ConfigLimpieza()
        if cfg.normalizar_unicode:
            texto = unicodedata.normalize("NFKC", texto)
        if cfg.a_minusculas:
            texto = texto.lower()
        if cfg.quitar_urls:
            texto = Tweet.URL.sub(" ", texto)
        if cfg.quitar_menciones:
            texto = Tweet.MENCION.sub(" ", texto)
        if cfg.quitar_hashtags:
            texto = Tweet.HASHTAG.sub(" ", texto)

        texto_limpio = Tweet.ESPACIO.sub(" ", texto).strip()
        return texto_limpio

    @staticmethod
    def tokenizar(texto: str) -> list[str]:
        return [t for t in Tweet.TT.tokenize(texto)]
    




# PIPELINE DE LIMPIEZA Y/O TOKENIZACIÓN DE TWEETS

In [7]:
# ===================== TRANSFORMACIONES EN PIPELINE =====================
"""
Recordemos que los pipelines de polars si es un lazyframe no se ejecutan hasta que se les pida con .collect() (Docoumentación)
"""


lf_registros_crudos = lf_registros


Logger.info("PIPELINE: LIMPIEZA/ TOKENIZACIÓN")
lf_limpio = (
    lf_registros_crudos.with_columns(
        pl.col("tweet_crudo")
          .map_elements(Tweet.limpiar)
          .alias("texto_limpio")
    )
)

Logger.debug(f"Mostrando registros del lazyframe limpio:\n {lf_limpio.limit(2).collect()}")

primer_tweet_limpio = (
    lf_limpio
    .select('texto_limpio')
    .limit(1)
    .collect()
    .to_series()
    .to_list()[0]
)

Logger.info(f"Primer tweet limpio: {primer_tweet_limpio}")
        
lf_tokens = (
    lf_limpio.with_columns(
        pl.col("texto_limpio")
          .map_elements(
              Tweet.tokenizar, 
              return_dtype=pl.List(pl.Utf8), 
              skip_nulls=True
            )
          .alias("tweet_tokenizado")
    )
)

Logger.debug(f"Mostrando registros del lazyframe tokenizado:\n {lf_tokens.limit(2).collect()}")


# ======================== Metodo para extraer el primer tweet tokenizado con polars =====================
primer_tweet_tokenizado = (
    lf_tokens
    .select("tweet_tokenizado") # Seleccionamos la columna que nos interesa
    .limit(1) 
    .collect()
    .item()  # Sirve para extraer el valor cuando el DataFrame es 1x1
)
# ========================================================================================================

Logger.info(f"Primer tweet tokenizado: {primer_tweet_tokenizado}")

for token in primer_tweet_tokenizado:
    Logger.info(f"Token: {token}")
    


[INFO] PIPELINE: LIMPIEZA/ TOKENIZACIÓN
[DEBUG] Mostrando registros del lazyframe limpio:
 shape: (2, 6)
┌──────────────────────┬──────────────────────┬──────────┬────────┬─────────┬──────────────────────┐
│ id_user              ┆ tweet_crudo          ┆ pais     ┆ genero ┆ idioma  ┆ texto_limpio         │
│ ---                  ┆ ---                  ┆ ---      ┆ ---    ┆ ---     ┆ ---                  │
│ str                  ┆ str                  ┆ str      ┆ str    ┆ str     ┆ str                  │
╞══════════════════════╪══════════════════════╪══════════╪════════╪═════════╪══════════════════════╡
│ 74bcc9b0882c8440716f ┆ Tiene que valer la   ┆ colombia ┆ female ┆ Español ┆ tiene que valer la   │
│ f370494aea…          ┆ pena que es…         ┆          ┆        ┆         ┆ pena que es…         │
│ 74bcc9b0882c8440716f ┆ Tintas chinas, si    ┆ colombia ┆ female ┆ Español ┆ tintas chinas, si    │
│ f370494aea…          ┆ ven ésto, es…        ┆          ┆        ┆         ┆ ven ésto,

# VAMOS A GENERAR UN VOCABULARIO GLOBAL

In [8]:
# ===================== VOCABULARIO GLOBAL =====================

Logger.info("CONSTRUCCIÓN DE VOCABULARIO GLOBAL")


lf_tokens_aplanados = (
    lf_tokens
    .select(
        'id_user', 
        pl.col('tweet_tokenizado')
    )
    .explode('tweet_tokenizado')
    .rename({'tweet_tokenizado':'tokens_del_tweet'})
)


lf_vocabulario = (
    lf_tokens_aplanados
    .group_by('tokens_del_tweet')
    .agg([
        pl.len().alias('frecuencia_global'),              # Conteo total de apariciones de cada token
        pl.n_unique('id_user').alias('documentos_unicos') # En cuántos documentos aparece ese token
    ])
)


Logger.debug(f"Mostrando registros del lazyframe tokenizado y aplanado:\n {lf_tokens_aplanados.collect()}")
Logger.debug(f"Mostrando vocabulario:\n {lf_vocabulario.collect()}")


[INFO] CONSTRUCCIÓN DE VOCABULARIO GLOBAL
[DEBUG] Mostrando registros del lazyframe tokenizado y aplanado:
 shape: (5_908_700, 2)
┌─────────────────────────────────┬──────────────────┐
│ id_user                         ┆ tokens_del_tweet │
│ ---                             ┆ ---              │
│ str                             ┆ str              │
╞═════════════════════════════════╪══════════════════╡
│ 74bcc9b0882c8440716ff370494aea… ┆ tiene            │
│ 74bcc9b0882c8440716ff370494aea… ┆ que              │
│ 74bcc9b0882c8440716ff370494aea… ┆ valer            │
│ 74bcc9b0882c8440716ff370494aea… ┆ la               │
│ 74bcc9b0882c8440716ff370494aea… ┆ pena             │
│ …                               ┆ …                │
│ 7ea28a182539c384d69cba3d10623f… ┆ recupere         │
│ 7ea28a182539c384d69cba3d10623f… ┆ ,                │
│ 7ea28a182539c384d69cba3d10623f… ┆ saludos          │
│ 7ea28a182539c384d69cba3d10623f… ┆ !                │
│ 7ea28a182539c384d69cba3d10623f… ┆ !        

# GENERAMOS LAS ETIQUETAS PARA PODER AVENTARNOS LO DEL ENTRENAMIENTO

In [9]:
# -------------------- Parámetros de reducción de dimensión --------------------
lf_documentos_unicos = (
    lf_limpio
    .select(['id_user', 'pais', 'genero'])
    .unique(maintain_order=True)
    .sort('id_user')
    .with_row_index(name='indice_doc', offset=0)
)


Logger.info(f"Total de documentos (usuarios): {lf_documentos_unicos.count().collect()[0,0]}")


# 2) Etiquetas alineadas (y_train)
lf_etiquetas = (
    lf_documentos_unicos
    .select('pais', 'genero')
    .collect()
    .to_series()
)

Logger.debug(f"Muestra de etiquetas (y): {lf_etiquetas}")


[INFO] Total de documentos (usuarios): 4200
[DEBUG] Muestra de etiquetas (y): shape: (4_200,)
Series: 'pais' [str]
[
	"spain"
	"peru"
	"venezuela"
	"peru"
	"spain"
	…
	"colombia"
	"chile"
	"peru"
	"peru"
	"peru"
]


In [10]:
# 3) Vocabulario reducido → diccionario de índices (token → columna)

MAX_TERMINOS = 50_000  
MIN_DOCS = 5

lf_vocabulario_reducido = (
    lf_vocabulario
    .filter(
        pl.col('documentos_unicos') >= MIN_DOCS)      # filtra tokens raros
    .sort('frecuencia_global', descending=True)
    
    
    #.limit(MAX_TERMINOS)                                 
)

Logger.debug(f"Mostrando vocabulario reducido:\n {lf_vocabulario_reducido.collect()}")  


lf_vocabulario_reducido_numpy =(
    lf_vocabulario_reducido
    .select('tokens_del_tweet')
    .collect()
    .to_numpy()
    .tolist()
)


Logger.debug(f"Vocabulario reducido numoy: {lf_vocabulario_reducido_numpy[:5]}")


tokens_ordenados = (
    lf_vocabulario_reducido
    .select('tokens_del_tweet')
)




# # Usado por el profe
# dicc_indices = {token: i for i, token in enumerate(tokens_ordenados)}

# for token, i in list(dicc_indices.items())[:10]:
#     Logger.info(f"Token: {token} → Índice columna: {i}")



# Lazy_frame de un dict_indices

lf_dict_indices = (
    tokens_ordenados
    .with_row_index(name='indice_col', offset=0)
)

Logger.debug(f"Mostrando diccionario de índices:\n {lf_dict_indices.collect()}")

[DEBUG] Mostrando vocabulario reducido:
 shape: (38_575, 3)
┌──────────────────┬───────────────────┬───────────────────┐
│ tokens_del_tweet ┆ frecuencia_global ┆ documentos_unicos │
│ ---              ┆ ---               ┆ ---               │
│ str              ┆ u32               ┆ u32               │
╞══════════════════╪═══════════════════╪═══════════════════╡
│ de               ┆ 215391            ┆ 4190              │
│ ,                ┆ 161864            ┆ 4125              │
│ .                ┆ 156802            ┆ 4005              │
│ que              ┆ 149950            ┆ 4155              │
│ la               ┆ 135889            ┆ 4183              │
│ …                ┆ …                 ┆ …                 │
│ posicionado      ┆ 5                 ┆ 5                 │
│ cerrarse         ┆ 5                 ┆ 5                 │
│ posan            ┆ 5                 ┆ 5                 │
│ valieron         ┆ 5                 ┆ 5                 │
│ rayar            ┆ 5   

In [11]:


"""
VERSION DEL PROFE
"""
    

# def built_bow_tr_profe_version(tweets, Vocabulario, dict_indices):
#     # Tweets: lista de tweets tokenizados | LazyFrame ya coleccionado
#     # Vocabulario: lista de tuplas (token, frecuencia) | LazyFrame ya coleccionado
#     # dict_indices: diccionario índice → token columna | lazyFrame ya coleccionado

#     BOW_FRECUENCIA = np.zeros((len(tweets), len(Vocabulario)), dtype=int)
#     BOW_BINARIO = np.zeros((len(tweets), len(Vocabulario)), dtype=int)

#     contador = 0
#     for tweet in tweets:
#         for palabra in tweet:
#             if palabra in dict_indices:
#                 Logger.debug(f"Palabra encontrada: {palabra} en tweet {contador}")
#                 BOW_FRECUENCIA[contador, dict_indices[palabra]] = tweet[palabra] # FRECUENCIA DE LA PALABRA EN VEZ DE HACERLO BINARIO
#                 BOW_BINARIO[contador, dict_indices[palabra]] = 1 # HACIENDOLO BINARIO
#         contador += 1

#     return BOW_FRECUENCIA, BOW_BINARIO




def built_bow_tr_profe_version(tweets_tokens, vocab_tokens, dict_indices):
    # Tweets: lista de tweets tokenizados | LazyFrame ya coleccionado
    # Vocabulario: lista de tuplas (token, frecuencia) | LazyFrame ya coleccionado
    # dict_indices: diccionario índice → token columna | lazyFrame ya coleccionado
    
    V = len(vocab_tokens)
    n = len(tweets_tokens)
    
    BOW_FRECUENCIA = np.zeros((n, V), dtype=int)
    BOW_BINARIO    = np.zeros((n, V), dtype=int)
    
    for i, tokens in enumerate(tweets_tokens):
        frec = Counter(tokens)              # {'de': 3, 'la': 1, ...}
        for token, frecuencia in frec.items():
            columna_vocab = dict_indices.get(token)       # columna del vocabulario
            if columna_vocab is not None:                    # solo si el token está en el vocab reducido
                BOW_FRECUENCIA[i, columna_vocab] = frecuencia
                BOW_BINARIO[i, columna_vocab] = 1
    return BOW_FRECUENCIA, BOW_BINARIO





# CONSTRUCCIÓN DE BOW (BAG OF WORDS) - MATRIZ DE TÉRMINO-RECURSO (TR)

In [ ]:
from collections import Counter

# --- A) Preparar insumos en los tipos correctos ------------------------------

# 1) vocabulario: aplanar si viene como [['de'], [','], ...]
vocab_tokens = (
    lf_vocabulario_reducido
    .select('tokens_del_tweet')
    .collect()
    .get_column('tokens_del_tweet')
    .to_list()
)
# Si insistieras en usar tu *_numpy:
# vocab_tokens = [x[0] for x in lf_vocabulario_reducido_numpy]

Logger.info(f"Vocabulario preparado: {len(vocab_tokens):,} términos")

# 2) dict_indices: convertir DataFrame -> dict {token: indice}
df_dict = lf_dict_indices.collect()  # columnas: ['indice_col','tokens_del_tweet']
dict_indices = dict(zip(
    df_dict.get_column('tokens_del_tweet').to_list(),
    map(int, df_dict.get_column('indice_col').to_list())
))
Logger.info(f"dict_indices listo: {len(dict_indices):,} entradas")

# 3) tweets: lista de listas de tokens
tweets = (
    lf_tokens
    .select("tweet_tokenizado")
    .limit(10000)  # solo si quieres probar con un subset
    .collect()
    .get_column("tweet_tokenizado")
    .to_list()
)
Logger.info(f"Tweets preparados: {len(tweets):,} documentos")


# --- B) Función del profe adaptada a listas de tokens + dict_indices ---------

def built_bow_tr_profe_version(tweets_tokens, vocab_tokens, dict_indices):
    V = len(vocab_tokens)
    n = len(tweets_tokens)
    BOW_FRECUENCIA = np.zeros((n, V), dtype=int)
    BOW_BINARIO    = np.zeros((n, V), dtype=int)

    for i, toks in enumerate(tweets_tokens):
        if not toks:
            continue
        frec = Counter(toks)  # {'de': 3, 'la': 1, ...}
        for token, cnt in frec.items():
            j = dict_indices.get(token)   # columna del vocabulario
            if j is not None:
                BOW_FRECUENCIA[i, j] = cnt
                BOW_BINARIO[i, j] = 1
        print_bar(i, n, contexto="Construyendo BOW")
    return BOW_FRECUENCIA, BOW_BINARIO


# --- C) Construir BOW con los insumos ya limpios -----------------------------
print()
BOW_FRECUENCIA, BOW_BINARIO = built_bow_tr_profe_version(tweets, vocab_tokens, dict_indices)
Logger.info("BOW construido.")

# ========== Chequeo rápido =========
nnz_freq = int((BOW_FRECUENCIA > 0).sum())
nnz_bin  = int(BOW_BINARIO.sum())
Logger.info(f"BOW frecuencia: {BOW_FRECUENCIA.shape} | nnz={nnz_freq:,}")
Logger.info(f"BOW binario:    {BOW_BINARIO.shape}    | nnz={nnz_bin:,}")
# ===================================

[INFO] Vocabulario preparado: 38,575 términos
[INFO] dict_indices listo: 38,575 entradas
[INFO] Tweets preparados: 10,000 documentos
Construyendo BOW: |🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩| 100.00%[INFO] BOW construido.
[INFO] BOW frecuencia: (10000, 38575) | nnz=115,921
[INFO] BOW binario:    (10000, 38575)    | nnz=115,921


In [13]:
"""
PROFE VERSION
"""
def compute_dor_profe(TR_densa: np.ndarray) -> np.ndarray:
    DTR = np.zeros((TR_densa.shape[1], TR_densa.shape[0]), dtype=float)
    tam_v = TR_densa.shape[1]
    for i, doc in enumerate(TR_densa):
        pos_no_cero = np.nonzero(doc)[0]
        tam_vocab_doc = len(pos_no_cero)
        for termino in pos_no_cero:
            DTR[termino, i] = doc[termino] * np.log(tam_v / max(1, tam_vocab_doc))
    return DTR


TR_frecuencia_densa = BOW_FRECUENCIA
DTR_frecuencia = compute_dor_profe(TR_frecuencia_densa)

TR_binaria_densa    = BOW_BINARIO
DTR_binaria    = compute_dor_profe(TR_binaria_densa)



Logger.info(f"DOR frecuencia: {DTR_frecuencia.shape} | DOR binaria: {DTR_binaria.shape}")

[INFO] DOR frecuencia: (38575, 10000) | DOR binaria: (38575, 10000)


In [ ]:
# ================== NORMALIZACIÓN L2 (TR y DOR) ==================
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

# 1) Pasar tus BOW densos a CSR (rápido y memoria-friendly)
TR_frecuencia_csr = csr_matrix(BOW_FRECUENCIA, dtype=np.float32)
TR_binaria_csr    = csr_matrix(BOW_BINARIO,    dtype=np.float32)
y_train = lf_etiquetas.to_list()

Logger.info(f"[CSR] TR_frecuencia: shape={TR_frecuencia_csr.shape}, nnz={TR_frecuencia_csr.nnz:,}")
Logger.info(f"[CSR] TR_binaria:    shape={TR_binaria_csr.shape}, nnz={TR_binaria_csr.nnz:,}")

# 2) Normalizar L2 por documento (filas) en TR
TR_frecuencia_L2_csr = normalize(TR_frecuencia_csr, norm='l2', copy=True)
TR_binaria_L2_csr    = normalize(TR_binaria_csr,    norm='l2', copy=True)

Logger.info("[L2] TR normalizado (frecuencia y binario) listo.")

# 3) (OPCIONAL) Normalizar L2 también los DOR del profe
#    DOR es término×documento → normalizamos por documento (columnas),
#    así que transponemos, normalizamos filas y des-transponemos.
#    Solo si ya creaste DTR_frecuencia / DTR_binaria (matrices densas).
if 'DTR_frecuencia' in globals():
    DOR_frecuencia_L2 = normalize(DTR_frecuencia.T, norm='l2', copy=True).T
    Logger.info("[L2] DOR_frecuencia normalizado.")
if 'DTR_binaria' in globals():
    DOR_binaria_L2 = normalize(DTR_binaria.T, norm='l2', copy=True).T
    Logger.info("[L2] DOR_binaria normalizado.")

# 4) Split 80/20 con TR (elige qué representación usar)

Xf_tr, Xf_va, y_tr, y_va   = train_test_split(TR_frecuencia_csr,    y_train, test_size=0.2, stratify=y_train, random_state=42)
Xb_tr, Xb_va, _, _         = train_test_split(TR_binaria_csr,       y_train, test_size=0.2, stratify=y_train, random_state=42)
Xf_trL2, Xf_vaL2, _, _     = train_test_split(TR_frecuencia_L2_csr, y_train, test_size=0.2, stratify=y_train, random_state=42)
Xb_trL2, Xb_vaL2, _, _     = train_test_split(TR_binaria_L2_csr,    y_train, test_size=0.2, stratify=y_train, random_state=42)

Logger.info(f"Shapes → Frecuencia tr={Xf_tr.shape} va={Xf_va.shape} | Binaria tr={Xb_tr.shape} va={Xb_va.shape}")

[INFO] [CSR] TR_frecuencia: shape=(10000, 38575), nnz=115,921
[INFO] [CSR] TR_binaria:    shape=(10000, 38575), nnz=115,921
[INFO] [L2] TR normalizado (frecuencia y binario) listo.
[INFO] [L2] DOR_frecuencia normalizado.
[INFO] [L2] DOR_binaria normalizado.


ValueError: Found input variables with inconsistent numbers of samples: [10000, 4200]